In [1]:
import pandas as pd
import numpy as np
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from nltk.corpus import stopwords
import re
import string
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors

In [2]:
client_data = pd.read_csv('./data/clients.csv',skipinitialspace = True)
student_group_data = pd.read_csv('./data/group.csv',skipinitialspace =True)

In [3]:
client_data.head(3)

,Company Name,Title,Project brief,Project Background,Requiements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Generates user interface events and observes t...
1,NaN,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,To propose the features in determining obesity...
2,NaN,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,to provide direct communications on employment...


In [4]:
student_group_data.head(4)

,group_id,group_details
0,1,Data Science Artificial Intelligence Machine L...
1,2,Full Stack Web Development Project Management ...


# Exploratory Data Analysis

In [5]:
client_data['Requirements'] = client_data['Project brief '].map(str) + ' ' + client_data['Project Background']+' '+client_data['Requiements']

In [6]:
client_data.head(3)


,Company Name,Title,Project brief,Project Background,Requiements,Requirements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Generates user interface events and observes t...,Automated testing suite for all software produ...
1,NaN,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,To propose the features in determining obesity...,Identify the effective indicators of obesity b...
2,NaN,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,to provide direct communications on employment...,To improve employee communications and engagem...


In [7]:
client_data = client_data.drop(['Requiements'],axis=1)

In [8]:
client_data = client_data.fillna(' ')

In [9]:
client_data.head()

,Company Name,Title,Project brief,Project Background,Requirements
0,ATX,ATX Software Testing Suite,Automated testing suite for all software produ...,This project will focus on graphical user inte...,Automated testing suite for all software produ...
1,,Prediction on Obesity associates with Lifestyl...,Identify the effective indicators of obesity b...,The main objective of the present study is to ...,Identify the effective indicators of obesity b...
2,,Mobile Advertising Platform with Designing Sui...,To improve employee communications and engagem...,The goal of the project is to create a mobile ...,To improve employee communications and engagem...
3,PurposePurchase Sdh Bhd,PurposePurchase,"\ncause-related ecommerce app, as well as a pl...",PurposePurchase is an online e-commerce platfo...,"\ncause-related ecommerce app, as well as a pl..."
4,,Educational Game,The aim of this project is to create an educat...,"The students will be given a concept document,...",The aim of this project is to create an educat...


In [10]:
client_data.shape

(25, 5)

In [11]:
stopwords_ = stopwords.words('english')
stopword_ = set(stopwords.words('english'))
wn = WordNetLemmatizer()

In [12]:
# Create word tokens
def token_txt(token):
    return token not in stopword_ and token not in list(string.punctuation) and len(token) > 2   
  

In [13]:
def clean_txt(text):
  clean_text = []
  clean_text2 = []
  text = re.sub("'", "", text)
  text = re.sub("(\\d|\\W)+", " ", text) 
  text = text.replace("nbsp", "")
  clean_text = [wn.lemmatize(word, pos = "v") for word in word_tokenize(text.lower()) if token_txt(word)]
  clean_text2 = [word for word in clean_text if token_txt(word)]
  return " ".join(clean_text2)

In [14]:
client_data['Requirements'] = client_data['Requirements'].apply(clean_txt)

In [15]:
client_data.tail()

,Company Name,Title,Project brief,Project Background,Requirements
20,,,,,
21,Agensi Pekerjaan & Latihan TAPRO Sdn Bhd,(CRM) tool for recruitment.,"Currently, selected information of the candida...",Aim to build a CRM tool for recruitment,currently select information candidates extrac...
22,People Systems Consultancy Sdn Bhd,App to build Financial Inclusion and Intellige...,Build the financial resilience and intelligenc...,seeking support and assistance to: \n(a) build...,build financial resilience intelligence differ...
23,,Statistics: Zero to Hundred,"In the world of education, e-learning platform...",The aim of this project is to create an e-lear...,world education learn platform essential make ...
24,,WhatsApp Messenger Forensic Analysis Based on ...,,,


## Student Group corpus

In [16]:
student_group_data['group_details '] = student_group_data['group_details '].apply(clean_txt)

In [17]:
## list column names in dataframe student_group_data ##

student_group_data.columns



Index(['group_id', 'group_details '], dtype='object')

# Term Ferquency - Inverse Document Frequency

In [18]:
tfidf_vect = TfidfVectorizer()

# Fitting and transforming the vector
tfidf_comb = tfidf_vect.fit_transform((client_data['Requirements'])) 
tfidf_comb

<25x626 sparse matrix of type '<class 'numpy.float64'>'
	with 921 stored elements in Compressed Sparse Row format>

In [19]:
user = 1
grp_index = np.where(student_group_data['group_id'] == user)[0][0]

In [20]:
student_group_data = student_group_data.iloc[[grp_index]]
student_group_data

,group_id,group_details
0,1,data science artificial intelligence machine l...


In [22]:
stud_group_rec = tfidf_vect.transform(student_group_data['group_details '])
cos_stud_group_rec = list(map(lambda x: cosine_similarity(stud_group_rec, x), tfidf_comb))
cos_stud_group_rec

[array([[0.04122973]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.32713553]]),
 array([[0.01863]]),
 array([[0.04414271]]),
 array([[0.]]),
 array([[0.13394251]]),
 array([[0.03807897]]),
 array([[0.09514322]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.15403788]]),
 array([[0.25628427]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.]]),
 array([[0.06743477]]),
 array([[0.18286495]]),
 array([[0.]])]

In [23]:
def get_recommendation(top, client_data, scores):
  recommendation = pd.DataFrame(columns = ['Group', 'Title',  'Company Name', 'Score'])
  count = 0
  for i in top:
      recommendation.at[count, 'Group'] = user
      recommendation.at[count, 'Title'] = client_data['Title'][i]
      recommendation.at[count, 'Company Name'] = client_data['Company Name'][i]
      recommendation.at[count, 'Score'] =  scores[count]
      count += 1
  return recommendation

In [26]:
top10_stud_group_rec = sorted(range(len(cos_stud_group_rec)), key = lambda i: cos_stud_group_rec[i], reverse = True)[:10]
top10_stud_group_rec 

[4, 17, 23, 16, 8, 10, 22, 6, 0, 9]

In [27]:
list_scores = [cos_stud_group_rec[i] for i in top10_stud_group_rec][i][0][0] for i in top10_stud_group_rec]
list_scores

[0.3271355309562686,
 0.2562842657573386,
 0.1828649456114235,
 0.1540378795303859,
 0.13394251295991239,
 0.09514321719145973,
 0.06743477176781525,
 0.04414271265454619,
 0.04122973019904442,
 0.03807896668505875]

In [29]:
get_recommendation(top10_stud_group_rec, client_data, list_scores)

,Group,Title,Company Name,Score
0,1,Educational Game,,0.327136
1,1,Cyber Threat Intelligence Mining Model,,0.256284
2,1,Statistics: Zero to Hundred,,0.182865
3,1,Hilti Global Territory Management Solution,Hilti,0.154038
4,1,\nNodejs Port of ATX Wallet System\n,ATX / MSC,0.133943
5,1,Online Assessment in Education System,,0.095143
6,1,App to build Financial Inclusion and Intellige...,People Systems Consultancy Sdn Bhd,0.067435
7,1,Content Management System in an IOT environment,Coinnov8 Web Portal,0.044143
8,1,ATX Software Testing Suite,ATX,0.04123
9,1,Nodejs Port of Mypospay Backend,ATX / MSC,0.038079


#  Top Recommendations with Count Vectorizer

In [30]:
count_vect = CountVectorizer()

# Fitting and transforming the vectorizer
count_comb = count_vect.fit_transform((client_data['Requirements'])) #fitting and transforming the vector
count_comb

<25x626 sparse matrix of type '<class 'numpy.int64'>'
	with 921 stored elements in Compressed Sparse Row format>

In [31]:
student_group_data = count_vect.transform(student_group_data['group_details '])
cv_cos_stud_data = list(map(lambda x: cosine_similarity(student_group_data, x), count_comb))

In [32]:
top10_cv_count = sorted(range(len(cv_cos_stud_data)), key = lambda i: cv_cos_stud_data[i], reverse = True)[:10]
list_cv_count = [cv_cos_stud_data[i][0][0] for i in top10_cv_count]
get_recommendation(top10_cv_count, client_data, list_cv_count)

,Group,Title,Company Name,Score
0,1,Educational Game,,0.387169
1,1,Cyber Threat Intelligence Mining Model,,0.285003
2,1,Statistics: Zero to Hundred,,0.210363
3,1,Hilti Global Territory Management Solution,Hilti,0.16462
4,1,\nNodejs Port of ATX Wallet System\n,ATX / MSC,0.152008
5,1,Online Assessment in Education System,,0.119808
6,1,App to build Financial Inclusion and Intellige...,People Systems Consultancy Sdn Bhd,0.065033
7,1,Nodejs Port of Mypospay Backend,ATX / MSC,0.056911
8,1,Content Management System in an IOT environment,Coinnov8 Web Portal,0.056478
9,1,ATX Software Testing Suite,ATX,0.051299


# Top Recommendations using KNN

In [33]:
n_neighbors = 11
KNN = NearestNeighbors(n_neighbors = n_neighbors, p = 2)
KNN.fit(tfidf_comb)

NearestNeighbors(n_neighbors=11)

In [36]:
knn_rec = KNN.kneighbors(stud_group_rec
                             , return_distance = True) 

In [37]:
knn_rec[0][0][1:]

array([1.        , 1.        , 1.        , 1.        , 1.        ,
       1.16005558, 1.219603  , 1.27838574, 1.30073988, 1.31609839])

In [38]:
top10_rec_knn = knn_rec[1][0][1:]
index_score_knn = knn_rec[0][0][1:]

get_recommendation(top10_rec_knn, client_data, index_score_knn)

,Group,Title,Company Name,Score
0,1,,,1.0
1,1,,,1.0
2,1,,,1.0
3,1,,,1.0
4,1,WhatsApp Messenger Forensic Analysis Based on ...,,1.0
5,1,Educational Game,,1.160056
6,1,Cyber Threat Intelligence Mining Model,,1.219603
7,1,Statistics: Zero to Hundred,,1.278386
8,1,Hilti Global Territory Management Solution,Hilti,1.30074
9,1,\nNodejs Port of ATX Wallet System\n,ATX / MSC,1.316098
